<a href="https://colab.research.google.com/github/AslanDevbrat/Seq2Seq/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [44]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, SimpleRNNCell, GRUCell, Dense, LSTMCell
from tensorflow.keras import Input

## Download the data


In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

## Configuration


In [6]:
!wget  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'
train_file_path = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_file_path= "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
test_file_path  = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"

--2022-06-17 14:16:57--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 173.194.192.128, 172.217.219.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   241MB/s    in 7.8s    

2022-06-17 14:17:05 (245 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [27]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = train_file_path


## Prepare the data


In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    #encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    #decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    #decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 100
Number of unique input tokens: 32
Number of unique output tokens: 25
Max sequence length for inputs: 15
Max sequence length for outputs: 20


## Build the model


In [74]:
 LSTMCell??

In [10]:
def lstm_cell():
  return tf.keras.layers.LSTMCell(latent_dim)

In [108]:
class Seq2seq(tf.keras.Model):
  def __init__(self, num_encoder_tokens, num_decoder_tokens,embedding_dim,num_of_layers,unit_type):
    super().__init__()
    self.encoder_inputs = Input(shape = (None, num_encoder_tokens))
    self.decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    self.vocab = num_encoder_tokens
    self.embedding_dim = embedding_dim
    self.num_decoder_tokens = num_decoder_tokens
    self.num_of_encoder_layer  =num_of_layers
    self.num_of_decoder_layer =num_of_layers
    self.type_encoder_unit =unit_type 
    self.type_decoder_unit =unit_type

  def get_embedding_layer(self, vocab, embedding_dim):
    return Embedding(vocab, embedding_dim)

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
    print(cell_type)
    if cell_type == "lstm":
      return LSTMCell(num_of_cell)
    elif cell_type == "rnn":
      return SimpleRNNCell(num_of_cell,)
    elif cell_type =="gru":
      return GRUCell(num_of_cell)
    else:
      print(f"Invalid cell type: {cell_type}")

  def get_encoder(self, cell_type = "lstm", num_of_layer = 1, latent_dim = 256, name = None):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim) for i in range(num_of_layer)]), return_state = True, name = name)

  def get_decoder(self, cell_type = "lstm", num_of_layer = 1, latent_dim = 256, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim,) for i in range(num_of_layer)]), return_sequences=True, return_state=True, name = name)

  def get_dense_layer(self, num_decoder_token, activation = "softmax"):
    return Dense(num_decoder_tokens, activation= activation)

  def train_step(self):
    # self.embedding_layer = self.get_embedding_layer( self.vocab, self.embedding_dim)
    # embedding_results = self.embedding_layer(self.encoder_inputs)

    self.encoder = self.get_encoder(self.type_encoder_unit, self.num_of_encoder_layer , 256, "encoder")
    encoder_results = self.encoder(self.encoder_inputs)

    self.encoder_outputs, self.encoder_states = encoder_results[0], encoder_results[1:]

    self.decoder = self.get_decoder(self.type_decoder_unit, self.num_of_decoder_layer, 256, "decoder")
    self.decoder_results = self.decoder(self.decoder_inputs, initial_state=self.encoder_states)

    self.decoder_output = self.decoder_results[0]
    self.decoder_dense = self.get_dense_layer(self.num_decoder_tokens)
    self.dense_output = self.decoder_dense(self.decoder_output)

  def build_model(self):
    self.model = keras.Model([self.encoder_inputs, self.decoder_inputs], self.dense_output)
    return self.model



In [109]:
seq2seq = Seq2seq(num_encoder_tokens, num_decoder_tokens, 1024,2,"lstm")
seq2seq.train_step()
seq2seq.build_model().summary()

lstm
lstm
lstm
lstm
Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_92 (InputLayer)          [(None, None, 32)]   0           []                               
                                                                                                  
 input_93 (InputLayer)          [(None, None, 25)]   0           []                               
                                                                                                  
 encoder (RNN)                  [(None, 256),        821248      ['input_92[0][0]']               
                                 [(None, 256),                                                    
                                 (None, 256)],                                                    
                                 [(None, 256),                         

In [69]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [lstm_cell() for _ in range(4)]), return_state = True)
#encoder = keras.layers.LSTM(latent_dim, return_state=True)
print(encoder(encoder_inputs))
encoder_results = encoder(encoder_inputs)
encoder_outputs, state = encoder_results[0], encoder_results[1:]

# We discard `encoder_outputs` and only keep the states.
encoder_states = state

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [lstm_cell() for _ in range(4)]), return_sequences=True, return_state=True)
decoder_results = decoder_lstm(decoder_inputs, initial_state=encoder_states)
#decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_results[0], decoder_results[1:]
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>, [<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>], [<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>], [<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>], [<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'rnn_18')>]]


In [12]:
encoder_inputs

<KerasTensor: shape=(None, None, 32) dtype=float32 (created by layer 'input_2')>

In [ ]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, None, 32)]   0           []                               
                                                                                                  
 input_55 (InputLayer)          [(None, None, 25)]   0           []                               
                                                                                                  
 rnn_43 (RNN)                   [(None, 256),        1871872     ['input_54[0][0]']               
                                 [(None, 256),                                                    
                                 (None, 256)],                                                    
                                 [(None, 256),                                             

In [ ]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, None, 32)]   0           []                               
                                                                                                  
 input_55 (InputLayer)          [(None, None, 25)]   0           []                               
                                                                                                  
 rnn_43 (RNN)                   [(None, 256),        1871872     ['input_54[0][0]']               
                                 [(None, 256),                                                    
                                 (None, 256)],                                                    
                                 [(None, 256),                                             

## Train the model


In [ ]:
batch_size = 3
sentence_max_length = 5
n_features = 2
new_shape = (batch_size, sentence_max_length, n_features)
x = tf.constant(np.reshape(np.arange(30), new_shape), dtype = tf.float32)

rnn_cells = [tf.keras.layers.LSTMCell(128) for _ in range(2)]
stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
lstm_layer = tf.keras.layers.RNN(stacked_lstm, return_state = True)

result = lstm_layer(x)
print(len(result))

3


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=2,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/2
27/27 [==============================] - 18s 401ms/step - loss: 0.9256 - accuracy: 0.1262 - val_loss: 1.7578 - val_accuracy: 0.2225
Epoch 2/2
27/27 [==============================] - 10s 377ms/step - loss: 0.8131 - accuracy: 0.1513 - val_loss: 1.7129 - val_accuracy: 0.2100


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
temp = model.layers[2].output
encoder_outputs, state = temp[0], temp[1:]  # lstm_1
encoder_states = state
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = state
decoder_lstm = model.layers[3]
temp = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_outputs, state_dec = temp[0], temp[1:]
decoder_states = state_dec
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        temp = decoder_model.predict([target_seq] + states_value)
        output_tokens, state = temp[0],temp[1:]

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = state
    return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: अं
Decoded sentence: anna

-
Input sentence: अंकगणित
Decoded sentence: annar

-
Input sentence: अंकल
Decoded sentence: anna

-
Input sentence: अंकुर
Decoded sentence: anna

-
Input sentence: अंकुरण
Decoded sentence: anna

-
Input sentence: अंकुरित
Decoded sentence: anna

-
Input sentence: अंकुश
Decoded sentence: anna

-
Input sentence: अंकुश
Decoded sentence: anna

-
Input sentence: अंग
Decoded sentence: annar

-
Input sentence: अंग
Decoded sentence: annar

-
Input sentence: अंगद
Decoded sentence: annar

-
Input sentence: अंगद
Decoded sentence: annar

-
Input sentence: अंगने
Decoded sentence: annar

-
Input sentence: अंगभंग
Decoded sentence: annar

-
Input sentence: अंगरक्षक
Decoded sentence: annar

-
Input sentence: अंगरक्षक
Decoded sentence: annar

-
Input sentence: अंगारा
Decoded sentence: annar

-
Input sentence: अंगारे
Decoded sentence: annar

-
Input sentence: अंगारे
Decoded sentence: annar

-
Input sentence: अंगी
Decoded sentence: annar

